# Playground with sampled data from KiT-RT

### Setup

In [ ]:
g = 1+1

In [ ]:
using KitBase, Plots, JLD2, Distributions, LinearAlgebra, Flux
using Flux: @epochs

In [ ]:
function regime_data(w, sw, f, u, K, Kn, μ=ref_vhs_vis(Kn, 1.0, 0.5), ω=0.81)
    gam = heat_capacity_ratio(K, 1)
    prim = conserve_prim(w, gam)
    Mu, Mxi, _, _1 = gauss_moments(prim, K)
    a = pdf_slope(prim, sw, K)
    swt = -prim[1] .* moments_conserve_slope(a, Mu, Mxi, 1)
    A = pdf_slope(prim, swt, K)
    tau = vhs_collision_time(prim, μ, ω)
    fr = chapman_enskog(u, prim, a, A, tau)
    L = norm((f .- fr) ./ prim[1])

    x = [w; sw; tau]
    y = ifelse(L <= 0.005, 0.0, 1.0)
    return x, y
end

function regime_number(Y, rg=0)
   idx = 0
    for i in axes(Y, 2)
       if Y[1, i] == rg
            idx += 1
        end
    end
    println("NS regime: $(idx) of $(size(Y, 2))")
    return nothing
end

function accuracy(nn, X, Z)
    Z1 = nn(X)

    ZA1 = [round(Z1[1, i]) for i in axes(Z1, 2)]
    ZA = [round(Z[1, i]) for i in axes(Z, 2)]

    accuracy = 0.0
    for i in eachindex(ZA)
        if ZA[i] == ZA1[i]
            accuracy += 1.0
        end
    end
    accuracy /= length(ZA)

    return accuracy
end

### Dataset

In [ ]:
file = open("pdfs.csv")
data = []
for line in eachline(file)
    a = split(line, ",")
    b = [parse(Float64, a[i]) for i = 2:length(a)]
    push!(data, b)
end
pdfs = data[3:end];

In [ ]:
#=file = open("../../../data/1d/a3_ev10.csv")
for line in eachline(file)
    a = split(line, ",")
    b = [parse(Float64, a[i]) for i = 2:length(a)]
    push!(data, b)
end
pdfs = [pdfs; data[3:end]]
=#
#=file = open("../../../data/1d/a8_ev5.csv")
for line in eachline(file)
    a = split(line, ",")
    b = [parse(Float64, a[i]) for i = 2:length(a)]
    push!(data, b)
end
pdfs = [pdfs; data[3:end]]=#
nd = length(pdfs) ÷ 2
pdfs

In [ ]:
vs = VSpace1D(-5.0, 5.0, length(data[1]), data[1], data[1][2:end] .- data[1][1:end-1], data[2])
δ = heaviside.(vs.u);

In [ ]:
dist = Uniform(0.005, 0.1)
dxs = rand(dist, nd)
dist = Uniform(0.0001, 1.0)
kns = rand(dist, nd);
dist = Uniform(0.1, 1.0)
rhos = rand(dist, nd);

In [ ]:
X = [1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0]
Y = [0.0]
for i = 1:nd
    try
        #fL = pdfs[i]; fR = pdfs[nd*2-i]
        fL = pop!(pdfs) .* pop!(rhos); fR = pop!(pdfs) .* pop!(rhos) # shuffle
        wL = moments_conserve(fL, vs.u, vs.weights); wR = moments_conserve(fR, vs.u, vs.weights)
        #@show wL, wR
        
        f = @. fL * δ + fR * (1.0 - δ)
        w = moments_conserve(f, vs.u, vs.weights)
        #f = @. (fL + fR) / 2
        #w = @. (wL + wR) / 2
        
        sw = @. (wR - wL) / dxs[i]
        
        tmpx, tmpy = regime_data(w, sw, f, vs.u, 0, kns[i])
        X = hcat(X, tmpx)
        Y = hcat(Y, tmpy)
    catch
    end
end

In [ ]:
regime_number(Y)

In [ ]:
idx = Int(floor(rand() * size(X, 2)))
plot(data[1], data[idx], ylabel="$(idx)-th pdf")

### Model

In [ ]:
@load "../nn_scalar.jld2" nn

In [ ]:
accuracy(nn, X, Y)

In [ ]:
data = Flux.Data.DataLoader((X, Y), shuffle = true)
ps = Flux.params(nn)
sqnorm(x) = sum(abs2, x)
#loss(x, y) = sum(abs2, nn(x) - y) / size(x, 2) #+ 1e-6 * sum(sqnorm, ps)
loss(x, y) = Flux.binarycrossentropy(nn(x), y)
cb = () -> println("loss: $(loss(X, Y))")
opt = ADAM()

In [ ]:
@epochs 2 Flux.train!(loss, ps, data, opt, cb = Flux.throttle(cb, 1))

In [ ]:
cd(@__DIR__)
@save "nn_rif.jld2" nn # reinforcement neural model

### Test

In [ ]:
accuracy(nn, X, Y)

In [ ]:
nn(X)

In [ ]:
X

In [ ]:
Y